In [1]:
!pip install transformers accelerate datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7

In [2]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.3 MB/s eta 0:00:00


In [3]:
!nvidia-smi

Sun Jul 14 20:32:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   49C    P8              13W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

### GPT

In [2]:
name = 'ai-forever/ruGPT-3.5-13B'
# name = 'ai-forever/mGPT-13B'
#name = 'ai-forever/rugpt3small_based_on_gpt2'

# Loading the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    name,
    device_map='cuda:0',
    load_in_8bit=True,
    offload_folder='./_tmp',
    max_memory={0: f'{int(torch.cuda.mem_get_info()[0] / 1024 ** 3) - 2}GB'},
    cache_dir="/content/.cache/huggingface"
)
tokenizer = AutoTokenizer.from_pretrained(name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
import pandas as pd

df = pd.read_csv(r'./stimuli_2018_test1.csv')[['Prot', 'Sent', 'Group', 'N1', 'N2', 'Pred', 'Code', 'Sentence']]
df = df.dropna(axis=0)

In [4]:
import datasets
import numpy as np
import torch
from torch.nn import CrossEntropyLoss
from transformers import AutoModelForCausalLM, AutoTokenizer

import evaluate
from evaluate import logging

from tqdm.auto import tqdm

In [5]:
df['leftGPT'] = df['Sentence'].apply(lambda a: ' '.join(a.strip().split()[:4]))
#df['rightA'] = df['Sentence'].apply(lambda a: ' '.join(a.strip().split()[3:]))

In [14]:
df.head()

,Prot,Sent,Group,N1,N2,Pred,Code,Sentence,leftGPT,p_left
0,1.0,1.0,1.0,S,S,S,S_S-S,Абонемент на концерт был дорогим из-за участия...,Абонемент на концерт был,-21.265625
1,2.0,1.0,1.0,S,S,P,P_S-S,Абонемент на концерт были дорогими из-за участ...,Абонемент на концерт были,-25.640625
2,3.0,1.0,1.0,S,P,S,S_S-P,Абонемент на концерты был дорогим из-за участи...,Абонемент на концерты был,-21.125000
3,4.0,1.0,1.0,S,P,P,P_S-P,Абонемент на концерты были дорогими из-за учас...,Абонемент на концерты были,-24.250000
4,5.0,1.0,1.0,P,S,S,S_P-S,Абонементы на концерт был дорогим из-за участи...,Абонементы на концерт был,-24.796875


In [11]:
def get_log_proba(text):
    #all_tokens = tokenizer(f', {text},', return_tensors='pt').input_ids.to('cuda')
    all_tokens = tokenizer(text, return_tensors='pt').input_ids.to('cuda')
    prev_tokens = all_tokens[:, :-1]
    next_tokens = all_tokens[:, 1:]
    with torch.inference_mode():
        model_out = model(prev_tokens)
        tokens_log_proba = torch.gather(torch.log_softmax(model_out.logits, -1), 2, next_tokens.unsqueeze(-1))
        phrase_log_proba = tokens_log_proba.sum().item()

        #loss = model_out.loss
        #perplexity = torch.exp(loss)
        #phrase_log_proba = perplexity.item()
    return phrase_log_proba

In [12]:
df['p_left'] = [get_log_proba(w) for w in tqdm(df['leftGPT'])]

  0%|          | 0/512 [00:00<?, ?it/s]

In [13]:
df[['Code','p_left']].groupby('Code')\
            .agg(['mean','std'])

p_left          
            mean       std
Code                      
P_P-P -22.236816  3.905305
P_P-S -21.910522  3.983962
P_S-P -27.466064  3.795282
P_S-S -26.174194  3.904622
S_P-P -27.290527  3.905805
S_P-S -26.571533  3.793163
S_S-P -22.659424  4.159971
S_S-S -20.198120  4.353169

# BERT

In [15]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("DeepPavlov/rubert-base-cased")

In [16]:
tokenizer("было", return_tensors="pt")

{'input_ids': tensor([[ 101, 3332,  102]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}

In [17]:
plural = 3294 #были
singular_m = 2067 #был
singular_f = 3370 #была
singular_i = 3332 #было

buli = {'были':3294,
        'был':2067,
        'была':3370,
        'было':3332,
        }

In [18]:
model.to('cuda')

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [19]:
inputs = tokenizer("Абонементы на концерт [MASK] дорогими .", return_tensors="pt").to('cuda')

mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

with torch.no_grad():
    logits = model(**inputs).logits

print('был --', logits[0, mask_token_index][0][singular_m])
print('были --', logits[0, mask_token_index][0][plural])

был -- tensor(6.7170, device='cuda:0')
были -- tensor(14.8506, device='cuda:0')


In [23]:
df['masked_left_BERT'] = df['Sentence'].apply(lambda a: ' '.join(a.strip().split()[:3])+' [MASK]')
#df['leftA'].apply(lambda a: a+' [MASK]' )

In [24]:
df.head()

,Prot,Sent,Group,N1,N2,Pred,Code,Sentence,leftGPT,p_left,masked_left_BERT
0,1.0,1.0,1.0,S,S,S,S_S-S,Абонемент на концерт был дорогим из-за участия...,Абонемент на концерт был,-21.265625,Абонемент на концерт [MASK]
1,2.0,1.0,1.0,S,S,P,P_S-S,Абонемент на концерт были дорогими из-за участ...,Абонемент на концерт были,-25.640625,Абонемент на концерт [MASK]
2,3.0,1.0,1.0,S,P,S,S_S-P,Абонемент на концерты был дорогим из-за участи...,Абонемент на концерты был,-21.125000,Абонемент на концерты [MASK]
3,4.0,1.0,1.0,S,P,P,P_S-P,Абонемент на концерты были дорогими из-за учас...,Абонемент на концерты были,-24.250000,Абонемент на концерты [MASK]
4,5.0,1.0,1.0,P,S,S,S_P-S,Абонементы на концерт был дорогим из-за участи...,Абонементы на концерт был,-24.796875,Абонементы на концерт [MASK]


In [26]:
plural = 3294 #были
singular_m = 2067 #был
singular_f = 3370 #была
singular_i = 3332 #было

def test_alt(sentence):
    inputs = tokenizer(sentence['masked_left_BERT'], return_tensors="pt").to('cuda')
    #print((inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0][0])
    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    #inputs = tokenizer(sentence['masked_full'], return_tensors="pt").to('cuda')

    with torch.no_grad():
        logits = model(**inputs).logits

    #print(logits[0, mask_token_index][0])

    return [logits[0, mask_token_index][0][plural].detach().cpu().numpy(),
            np.max([
            logits[0, mask_token_index][0][singular_m].detach().cpu().numpy(),
            logits[0, mask_token_index][0][singular_f].detach().cpu().numpy(),
            logits[0, mask_token_index][0][singular_i].detach().cpu().numpy()
            ]),]

dd = df.apply(lambda a: test_alt(a), axis=1 )

In [27]:
df['left_bert_is_singular'] = [i<j for i,j in dd]

In [28]:
df['left_bert_is_singular_score'] = [float(i[1]) for i in dd]
df['left_bert_is_plural_score'] = [float(i[0]) for i in dd]

In [29]:
df.head()

,Prot,Sent,Group,N1,N2,Pred,Code,Sentence,leftGPT,p_left,masked_left_BERT,left_bert_is_singular,left_bert_is_singular_score,left_bert_is_plural_score
0,1.0,1.0,1.0,S,S,S,S_S-S,Абонемент на концерт был дорогим из-за участия...,Абонемент на концерт был,-21.265625,Абонемент на концерт [MASK],True,3.069234,-3.261937
1,2.0,1.0,1.0,S,S,P,P_S-S,Абонемент на концерт были дорогими из-за участ...,Абонемент на концерт были,-25.640625,Абонемент на концерт [MASK],True,3.069234,-3.261937
2,3.0,1.0,1.0,S,P,S,S_S-P,Абонемент на концерты был дорогим из-за участи...,Абонемент на концерты был,-21.125000,Абонемент на концерты [MASK],True,2.110909,-1.037887
3,4.0,1.0,1.0,S,P,P,P_S-P,Абонемент на концерты были дорогими из-за учас...,Абонемент на концерты были,-24.250000,Абонемент на концерты [MASK],True,2.110909,-1.037887
4,5.0,1.0,1.0,P,S,S,S_P-S,Абонементы на концерт был дорогим из-за участи...,Абонементы на концерт был,-24.796875,Абонементы на концерт [MASK],False,-0.349254,2.868716
